## Import related functions

In [1]:
import cobra
import re 
import pandas as pd
#from script.ECMpy_function import *
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Input and output files

In [2]:
#Initial parameters
f = 0.406 
ptot = 0.56 
sigma = 1
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3) 
EC_max_file='./data/EC_kcat_max.json'# https://www.brenda-enzymes.org/brenda_download/file_download.php
method='AutoPACMEN'#DLKcat
use_substrate='EX_glc__D_e'
concentration=10
obj='BIOMASS_Ec_iML1515_core_75p37M'# CG_biomass_cgl_ATCC13032 EX_lys_L_e

#Originl ecmodel and result file
ecModel_file="./model/iML1515_irr_enz_constraint.json"
fluxes_infile_ori = './analysis/ECMpy_solution_BIOMASS_Ec_iML1515_core_75p37M_pfba.csv'
reaction_kcat_MW_file = "./analysis/get_kcat_mw_by_%s/reaction_kcat_MW.csv"%method
need_change_reaction_list=[]
changed_reaction_list=[]
round_num=1
reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#ecmodel and result file
json_output_file = './model/iML1515_irr_enz_constraint_adj.json'
reaction_kcat_MW_outfile = './analysis/get_kcat_mw_by_%s/reaction_change_by_enzuse.csv'%method

# Calibration kcat according Enzyme usage 

In [3]:
#Calibration
enz_model_obj=0
while (enz_model_obj<0.66 and round_num<50):#maxium growth rate
    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(ecModel_file,fluxes_infile_ori,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
    enz_model.objective=obj
    #change original substrate in model
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
            
    enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

    enz_model_obj=enz_model.slim_optimize()
    print('Calibration round %s : '%round_num+str(enz_model_obj))
    round_num=round_num+1
    
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)

Need changing reaction: 
PRFGS
Changed reaction: 
['PRFGS']
Calibration round 1 : 0.23173512444593233
Need changing reaction: 
KARA1_reverse
Changed reaction: 
['PRFGS']
Calibration round 2 : 0.23173512444593233
Need changing reaction: 
FBA_num1
Changed reaction: 
['PRFGS', 'FBA_num1']
Calibration round 3 : 0.24483790513370257
Need changing reaction: 
PFL_num3
Changed reaction: 
['PRFGS', 'FBA_num1', 'PFL_num3']
Calibration round 4 : 0.24723409719596082
Need changing reaction: 
3OAS121
Changed reaction: 
['PRFGS', 'FBA_num1', 'PFL_num3', '3OAS121']
Calibration round 5 : 0.26291740157273025
Need changing reaction: 
3OAS161
Changed reaction: 
['PRFGS', 'FBA_num1', 'PFL_num3', '3OAS121', '3OAS161']
Calibration round 6 : 0.2806003508042417
Need changing reaction: 
3OAS141
Changed reaction: 
['PRFGS', 'FBA_num1', 'PFL_num3', '3OAS121', '3OAS161', '3OAS141']
Calibration round 7 : 0.29935260943266906
Need changing reaction: 
GLUDy_reverse
Changed reaction: 
['PRFGS', 'FBA_num1', 'PFL_num3', '